# JEM092 Asset Pricing - Homework 2

<span style='background:lightblue'> **Group : 20107894 ;  
Assigned Factor : ILLIQ.**</span>  

Team Member :

Yanqi Guan : 20107894@fsv.cuni.cz

Lin Zhang : 15845542@fsv.cuni.cz

Jiayi Zeng : 46566215@fsv.cuni.cz


## Empirical Problem
In the homework you will demonstrate all the theoretical knowledge and empirical skills gained through the semester. Using financial market data you will 
* sort stocks into various portfolios, 
* examine time-series properties of the formed portfolios, 
* examine cross-sectional variation in portfolios and comment on importance of the sorting variables,  
* use the Fama-MacBeth regressions and Fama-French-Carhart-Pastor-Stambaugh factor model to assess the performance of the sorting variables

In the portfolio analysis you will use 3 sorting variables to create various portfolios. These variables are following:
* Beta
* Size
* one of the Book-to-Market ratio/Momentum/Short-term-reversal/Illiquidity/Skewness/Volatility
     + to each group the sorting variable was randomly assigned using the following code

Besides the firm level data you will need some of the factors often used in empirical asset pricing literature. The Fama-French 3 factors, Momentum and Short-term reversal can be downloaded from the Kenneth French web (https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) - file with FF three factors contain also risk-free rate. The liquidity factor can be downloaded from the web of Ľuboš Pástor (https://faculty.chicagobooth.edu/lubos-pastor/data)


# Solutions:


* **Environment setup**

In [46]:
# Setup environment
Sys.setenv(LANG = "en")
Sys.setlocale("LC_TIME", "English")
options(warn = -1)  # suppressing warnings
version[['version.string']]

if (!require(quantmod)) install.packages('quantmod')
if (!require(lubridate)) install.packages('lubridate')
if (!require(sandwich)) install.packages('sandwich')
if (!require(moments)) install.packages('moments')

library(quantmod)
library(lubridate)
library(sandwich)
library(moments)

[1] "English_United States.1252"

[1] "R version 4.1.2 (2021-11-01)"

In [47]:
# Load to continue ...
# save.image(file = "AP_Hw2.RData")
# load("AP_Hw2.RData")
# print("Welcome back!")

**This homework is coded in environment of R version 4.1.2, please note that running the code on different R version might get slightly different results.  
"Restart & Run ALL" code would take around <span style='background: lightblue'>3 minutes</span>.**

# 1. Data Preparation

In each month to obtain estimates of Beta, Illiquidity, total Skewness and total Volatilit use last 12 months of daily data. Further, for Beta estimation use data from Kenneth French library, i.e. Mkt-RF as market risk premium. 

* Sample period: 01.01.2007 - 31.12.2020
* Use data stored in the Moodle, from file "Asset_Pricing_HW_2_data.RData"

In [3]:
# load data from Moodle
load("Asset_Pricing_HW_2_data.RData")
ls()
# Read assigned symbols from file 
smbs <- read.csv('20107894_data_download.csv',colClasses = "character")
p_symbols <- smbs[[2]]
length(p_symbols)
head(p_symbols)

[1] "book_value_sap500" "MktCap_sap500"     "OHLCV_sap500"     
[4] "symbols"

[1] 250

[1] "F"   "CSX" "OKE" "DFS" "ITW" "MRO"

In [4]:
# set data range from 2007 - 2020
OHLCV <- OHLCV_sap500[p_symbols]
OHLCV <- lapply(p_symbols, function(y){OHLCV[[y]]["2007/2020"]})
names(OHLCV) <- p_symbols

MktCap <- MktCap_sap500[p_symbols]
MktCap <- lapply(p_symbols, function(y){MktCap[[y]]["2007/2020"]})
names(MktCap) <- p_symbols

book_value <- book_value_sap500[p_symbols]
book_value <- lapply(p_symbols, function(y){book_value[[y]]["2007/2020"]})
names(book_value) <- p_symbols

# check data loaded, we only need stock price and MktCap data
print(paste("OHLCV:", length(OHLCV)))
lapply(OHLCV, head, 2)[1:2]
print(paste("MktCap:", length(MktCap)))
lapply(MktCap, head, 2)[1:2]
print(paste("Value:", length(book_value)))
lapply(book_value, head, 2)[1:2]

[1] "OHLCV: 250"


$F
           F.Open F.High F.Low F.Close F.Volume F.Adjusted
2007-01-03   7.56   7.67  7.44    7.51 78652200   5.039420
2007-01-04   7.56   7.72  7.43    7.70 63454900   5.166914

$CSX
           CSX.Open CSX.High  CSX.Low CSX.Close CSX.Volume CSX.Adjusted
2007-01-03 3.862222 3.923333 3.805556  3.860000   57204900     2.924608
2007-01-04 3.865556 3.894444 3.828889  3.888889   36793800     2.946497


[1] "MktCap: 250"


$F
            [,1]
2007-01-03 14.21
2007-01-04 14.57

$CSX
            [,1]
2007-01-03 15.20
2007-01-04 15.32


[1] "Value: 250"


$F
           Stock Price Book Value per Share Price/Book Ratio
2007-03-31        5.29                -1.97            -2.68
2007-06-30        6.32                -0.93            -6.82

$CSX
           Stock Price Book Value per Share Price/Book Ratio
2007-03-31        3.40                 2.32             1.47
2007-06-30        3.83                 2.34             1.64


### <span style='background: lightblue'>1.1. loading Fama-French data</span>

* **Download Fama-French 3 factors, Momentum and Liquidity factor data from websites**

In [5]:
# Download factor data.

# Download daily FF3 factors.
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_TXT.zip",
    destfile = "F-F_Research_Data_Factors_daily.zip"
)
unzip("F-F_Research_Data_Factors_daily.zip")

# Download monthly FF3 factors.
# https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip",
    destfile = "F-F_Research_Data_Factors.zip"
)
unzip("F-F_Research_Data_Factors.zip")

# Download Developed Momentum Factor (Mom).
# https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_developed_mom.html
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/Developed_Mom_Factor_TXT.zip",
    destfile = "Developed_Mom_Factor_TXT.zip"
)
unzip("Developed_Mom_Factor_TXT.zip")

# Download liquidty data(liq).
download.file(
    "https://faculty.chicagobooth.edu/-/media/faculty/lubos-pastor/data/liq_data_1962_2021.txt",
    destfile = "liq_data_1962_2021.txt"
)

print("All files are downloaded successfully!")

# LIQUIDITY FACTORS OF PASTOR AND STAMBAUGH (JPE 2003) UPDATED THROUGH DEC 2021  
# Column 1: Month  
# Column 2: Levels of aggregated liquidity (Figure 1 in the paper)  
# Column 3: Innovations in aggregated liquidity (non-traded liquidity factor; equation(8); the main series)  
# Column 4: Traded liquidity factor (LIQ_V, 10-1 portfolio return)  
# Note: The traded factor is the value-weighted return on the 10-1 portfolio from a sort on historical liquidity betas. 
# This procedure is simpler than sorting on predicted betas (as in the original study) and through 2021 
# it is similarly successful at creating a spread in post-ranking betas. The traded factor has a positive and 
# significant alpha through 2021, consistent with liquidity risk being priced.

[1] "All files are downloaded successfully!"


**Load factor data from web**

In [6]:
# Load daily data
ff3_factors <- read.delim(
    'F-F_Research_Data_Factors_daily.txt',
    col.names = c('t', 'mkt_rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 24957,
    header = FALSE,
    skip = 5,
    stringsAsFactors = FALSE
)
ff3_factors[['t']] <- as.Date(as.character(ff3_factors[['t']]), '%Y%m%d')
ff3_factors <- as.xts(ff3_factors[, 2:5], order.by = ff3_factors[['t']])
# set data range from 2007 - 2020
ff3_factors <- ff3_factors["2007/2020"]
ff3_factors <- ff3_factors / 100

# Load monthly data
monthly_ff3_factors <- read.delim(
    'F-F_Research_Data_Factors.txt',
    col.names = c('t', 'mkt.rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 1137,
    header = FALSE,
    skip = 4,
    stringsAsFactors = FALSE
)

monthly_ff3_factors[['t']] <- as.Date(paste0(as.character(monthly_ff3_factors[['t']]), '01'), '%Y%m%d')
monthly_ff3_factors <- as.xts(monthly_ff3_factors[, 2:5], order.by = monthly_ff3_factors[['t']])
# set data range from 2007 - 2020
monthly_ff3_factors <- monthly_ff3_factors["2007/2020"]
monthly_ff3_factors <- monthly_ff3_factors / 100
# update date format as monthly, date as the last day of the monthly
index(monthly_ff3_factors) <- as.Date(as.yearmon(index(monthly_ff3_factors)), frac = 1)

# Load Mom data
monthly_mom_factors <- read.delim(
    'Developed_Mom_Factor.txt', 
    col.names = c('t', 'wml'),
    sep = '',
    nrows = 374,
    header = FALSE,
    skip = 7,
    stringsAsFactors = FALSE
)

monthly_mom_index <- as.Date(paste0(as.character(monthly_mom_factors[['t']]), '01'), '%Y%m%d')
monthly_mom_xts <- as.xts(monthly_mom_factors[, 2], order.by = monthly_mom_index)
# set data range from 2007 - 2020
monthly_mom_factors <- monthly_mom_xts["2007/2020"]/100
# update date format as monthly, date as the last day of the monthly
index(monthly_mom_factors) <- as.Date(as.yearmon(index(monthly_mom_factors)), frac = 1)
colnames(monthly_mom_factors) <-"mom"

# Load illiq data
liq_data <- read.delim(
    'liq_data_1962_2021.txt',
    col.names = c('t', 'Agg.Liq', 'Innov.Liq', 'Traded.Liq'),
    sep = '', nrows = 713, header = FALSE,
    skip = 11, stringsAsFactors = FALSE
)
#  we only need column 4 for Liquidity excess returns
monthly_liq_index <- as.Date(paste0(as.character(liq_data[['t']]), '01'), '%Y%m%d')
monthly_liq_xts <- as.xts(liq_data[, 4], order.by = monthly_liq_index)
# set data range from 2007 - 2020
monthly_liq_factors <- monthly_liq_xts["2007/2020"]
# update date format as monthly, date as the last day of the monthly
index(monthly_liq_factors) <- as.Date(as.yearmon(index(monthly_liq_factors)), frac = 1)
colnames(monthly_liq_factors) <-"liq"
factor_data <- cbind(monthly_ff3_factors, monthly_mom_factors,monthly_liq_factors)

# check data
head(factor_data,3)
tail(factor_data,3)

            mkt.rf    smb     hml     rf     mom        liq
2007-01-31  0.0140 0.0014 -0.0070 0.0044  0.0065 0.00375580
2007-02-28 -0.0196 0.0118 -0.0012 0.0038 -0.0004 0.02496826
2007-03-31  0.0068 0.0015 -0.0094 0.0043  0.0180 0.02931590

            mkt.rf    smb     hml    rf     mom        liq
2020-10-31 -0.0210 0.0440  0.0414 1e-04 -0.0169 0.04065628
2020-11-30  0.1247 0.0563  0.0210 1e-04 -0.1092 0.02308695
2020-12-31  0.0463 0.0483 -0.0141 1e-04 -0.0009 0.05024218

**Now we have the excess return spread data for all FFCPS factors, we will use those data for our FFCPS alpha calculation and Fama-MacBeth regression.** 

### <span style='background: lightblue'>1.2. Preparing stock price data and stock returns</span>

**preparing stock price data**

In [7]:
prices <- c()
bm_ratios  <- c()
for (i in 1:length(p_symbols)) {
    ticker <- p_symbols[i]
    # only use Adjusted price as stock price
    stock_prices = OHLCV[[ticker]][, 6]
    stock_bm_ratios <- book_value[[i]][, 2] / book_value[[i]][, 1]    
    colnames(stock_prices) <- ticker
    colnames(stock_bm_ratios) <- ticker    
    prices <- cbind(prices, stock_prices)
    bm_ratios <- cbind(bm_ratios, stock_bm_ratios)        
}

prices[1:2,1:5]
tail(prices,2)[,1:5]

                  F      CSX      OKE DFS      ITW
2007-01-03 5.039420 2.924608 8.800058  NA 31.89644
2007-01-04 5.166914 2.946497 8.708157  NA 32.09106

                  F      CSX      OKE      DFS      ITW
2020-12-30 8.770518 29.77474 34.89088 87.83402 198.0087
2020-12-31 8.701225 29.91318 35.12885 89.04384 199.6835

**Calculating stock returns**

Use dailyReturn and monthlyReturn functions to calculate arithmetic returns

In [8]:
# Compute daily and monthly returns.
daily_returns <- lapply(prices, dailyReturn, USE.NAMES = TRUE)
monthly_returns <- lapply(prices, monthlyReturn, USE.NAMES = TRUE)

daily_returns <- do.call('cbind', daily_returns)
monthly_returns <- do.call('cbind', monthly_returns)
colnames(daily_returns) <- colnames(prices)
colnames(monthly_returns) <- colnames(prices)

index(monthly_returns) <- as.Date(as.yearmon(index(monthly_returns)), frac = 1)
daily_returns[1:2, 1:5]
monthly_returns[1:2, 1:5]
# not all stocks have price on 2007, but they all have data on 2020

                    F         CSX         OKE DFS         ITW
2007-01-03 0.00000000 0.000000000  0.00000000  NA 0.000000000
2007-01-04 0.02529934 0.007484422 -0.01044323  NA 0.006101685

                     F        CSX          OKE DFS        ITW
2007-01-31  0.08255672 0.05900996  0.003775543  NA 0.11113562
2007-02-28 -0.02706042 0.02567941 -0.029130506  NA 0.01274734

Use the Risk Free rate from FF3 to calculate daily and monthly excess returns.

In [9]:
head(ff3_factors[, 4],2)
head(monthly_ff3_factors[, 4],2)

# Compute daily and monthly excess returns by minus risk free rate from ff3
# xts object could minus each other based on time stamps
for (i in 1:length(p_symbols)) {    
    daily_returns[, i] <- daily_returns[, i] - ff3_factors[, 4]
    monthly_returns[, i] <- monthly_returns[, i] - monthly_ff3_factors[, 4]
}

daily_returns[1:2, 1:5]
monthly_returns[1:2, 1:5]

                rf
2007-01-03 0.00022
2007-01-04 0.00022

               rf
2007-01-31 0.0044
2007-02-28 0.0038

                     F          CSX         OKE DFS          ITW
2007-01-03 -0.00022000 -0.000220000 -0.00022000  NA -0.000220000
2007-01-04  0.02507934  0.007264422 -0.01066323  NA  0.005881685

                     F        CSX           OKE DFS         ITW
2007-01-31  0.07815672 0.05460996 -0.0006244567  NA 0.106735618
2007-02-28 -0.03086042 0.02187941 -0.0329305056  NA 0.008947344

### <span style='background: lightblue'>1.3. Preparing beta</span>

**Use the CAPM model to estimate betas**

<span style='background: lightyellow'>This step takes 1 minutes</span>.

In [11]:
# Initialize an xts object for storing monthly_betas.
monthly_betas <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_betas) <- names(monthly_returns)


# every month should have 20 trading days, if available data fewer than
# 15 samples, meaning only less than 75% data, then ingore that stock.
check_data <- 12 * 15

# Iterate over rows and compute monthly_betas.
# i is months, j is stocks
system.time(
    for (i in 1:nrow(monthly_betas)) {
        # each i is for every month in data range
        # select data range, look back 12 months daily returns
        # put data need to evaluate into variable returns
        # and market_returns
        end_day <- index(monthly_betas[i])
        start_day <- end_day %m-% months(12)
        dates <- paste0(start_day, '/', end_day)
        returns <- daily_returns[dates]
        market_returns <- ff3_factors[dates]

        # Estimate monthly_betas for each stock in 250 stock list
        for (j in 1:ncol(monthly_betas)) {
            stock_returns <- returns[, j]
            # Merge excess returns and market excess returns.
            # xts matches by date in cbind
            model_data <- cbind(stock_returns, market_returns[, 1])
            # ingore NA
            model_data <- na.omit(model_data)
            # Don't estimate monthly_betas if observations is less than 200.
            if (nrow(model_data) < check_data) {next}        
            # call regression to calculate beta, use stock returns - market returns
            model_betas <- lm(model_data[, 1] ~ model_data[, 2])
            # Save the given beta to `monthly_betas`.
            # save the beta information at coefficients[[2]]
            monthly_betas[i, j] <- model_betas$coefficients[[2]]
        }    
    }
)

# The estimation of monthly_betas is based on the last 12 months of daily returns.
monthly_betas[15:17,1:5]

   user  system elapsed 
  61.52    0.00   61.55 

                  F      CSX       OKE      DFS       ITW
2008-03-31 1.122713 1.313259 0.6756543 2.060537 0.8624013
2008-04-30 1.145374 1.308042 0.6748264 2.067974 0.9004992
2008-05-31 1.155006 1.296945 0.6463733 2.088052 0.9020326

### <span style='background: lightblue'>1.4. Preparing size factor</span>

In [12]:
market_caps <- do.call(merge, MktCap)
colnames(market_caps) <- names(MktCap)

# Compute monthly market caps and sizes. MktCap
sizes <- log(market_caps)
monthly_market_caps <- apply.monthly(market_caps, tail, 1)
monthly_sizes <- apply.monthly(sizes, tail, 1)
# Replace -Inf and Inf in `monthly_sizes` with NAs.
monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA)))
index(monthly_market_caps) <- as.Date(as.yearmon(index(monthly_market_caps)), frac = 1)
index(monthly_sizes) <- as.Date(as.yearmon(index(monthly_sizes)), frac = 1)
                              
monthly_market_caps[1:2, 1:5]
monthly_sizes[1:2, 1:5]

               F   CSX  OKE DFS   ITW
2007-01-31 15.38 16.10 4.76  NA 28.50
2007-02-28 14.97 16.46 4.62  NA 28.87

                  F      CSX      OKE DFS      ITW
2007-01-31 2.733068 2.778819 1.560248  NA 3.349904
2007-02-28 2.706048 2.800933 1.530395  NA 3.362803

### <span style='background: lightblue'>1.5. Preparing illiquidity factor</span>

* **Calculate illiquidity factor**

**Calculate absolute return to nominal ratios by using Amihud(2002) foluma:**  
$$Illiq_i= \frac{1}{D} \sum_{d=1}^{D} \frac{\left\lvert R_{i,d} \right\rvert } {VOLD_{i,d}} $$
**This fomula calculates illiquidity as the ratio of the absolute value of the daily security return divided by the
daily dollar volume traded in the security, averaged over all days in the estimation period.**

In [13]:
abs_return_to_nominal_ratios <- c()

for (i in 1:length(p_symbols)) {
    ticker <- p_symbols[i]
    stock_data <- OHLCV[[ticker]]

    # get absolute nominals
    stock_abs_return_to_nominal_ratios <- abs(daily_returns[, 
        ticker]) / (stock_data[, 5] * stock_data[, 6] / 1000000)
    
    colnames(stock_abs_return_to_nominal_ratios) <- ticker
    abs_return_to_nominal_ratios <- cbind(abs_return_to_nominal_ratios, 
        stock_abs_return_to_nominal_ratios)
}

abs_return_to_nominal_ratios[1:2, 1:5]

                      F          CSX          OKE DFS          ITW
2007-01-03 5.550489e-07 1.314988e-06 9.492341e-06  NA 2.997272e-06
2007-01-04 7.649265e-05 6.700704e-05 8.649851e-04  NA 7.349766e-05

* **Compute illiquidity**

**Because the distribution of Illiq is highly skewed, so we use log-transformed versions of the different illiquidity variables, so define the Illiq factor as the natural log of one plus $Illiq ^ {1M}$.**

In [14]:
# Initialize an xts object for storing illiquidities.
# just create empty frame 

monthly_illiquidities <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_illiquidities) <- names(monthly_returns)

# Iterate over rows and compute illiquidities.
# i is every monthly from 2007-2020
for (i in 1:nrow(monthly_illiquidities)) {
    end_day <- index(monthly_illiquidities[i])
    # use last 12 months of daily data
    start_day <- end_day %m-% months(12)
    dates <- paste0(start_day, '/', end_day)
    # get daily ratio data from the date range
    ratios <- abs_return_to_nominal_ratios[dates]  

    # Compute illiquidities for each stock.
    for (j in 1:ncol(monthly_illiquidities)) {
        stock_ratios <- ratios[, j]

        # Save the given illiquidity to `monthly_illiquidities`.
        # use "natural log of one plus the avg 1 month Illiq of 12 months
        monthly_illiquidities[i, j] <- log(1 + mean(stock_ratios))
    }
}

monthly_illiq <- monthly_illiquidities * 10000                                 
                              
# 2007-2020, 14 years has 168 months
nrow(monthly_illiq)
monthly_illiq[1:2, 1:5]                             
                                      

[1] 168

                   F       CSX      OKE DFS       ITW
2007-01-31 0.4007055 1.0406651 5.565255  NA 0.7196594
2007-02-28 0.5105019 0.8774579 4.963222  NA 0.8972203

### <span style='background: lightblue'>1.6. Preparing value factor</span>

In [15]:
# Initialize an xts object for storing book-to-market ratios.
monthly_bm_ratios <- as.xts(
    matrix(
        nrow = nrow(bm_ratios),
        ncol = ncol(bm_ratios)
    ),
    order.by = index(bm_ratios)
)
names(monthly_bm_ratios) <- names(bm_ratios)

start_year <- as.numeric(format(min(index(monthly_bm_ratios)), '%Y'))
end_year <- as.numeric(format(max(index(monthly_bm_ratios)), '%Y'))

# Iterate over rows and compute BM ratios.
for (i in (start_year + 2):end_year) {
    start_day <- as.Date(paste0(i - 1, '06', '30'), format = '%Y%m%d')
    end_day <- as.Date(paste0(i, '05', '31'), format = '%Y%m%d')
    dates <- paste0(start_day, '/', end_day)
    end_of_previous_year <- as.Date(paste0(i - 2, '12', '31'), format = '%Y%m%d')
    for (j in 1:ncol(bm_ratios)) {
        monthly_bm_ratios[dates, j] <- as.numeric(bm_ratios[end_of_previous_year, j][[1]])
    }
}

monthly_bm_ratios[31:33,1:5]

                   F       CSX      OKE DFS       ITW
2009-07-31 -3.935065 0.8156028 2.372057  NA 0.6031373
2009-08-31 -3.935065 0.8156028 2.372057  NA 0.6031373
2009-09-30 -3.935065 0.8156028 2.372057  NA 0.6031373

### <span style='background: lightblue'>1.7. Preparing momentum factor</span>

In [16]:
# Initialize an xts object for storing momentum.
monthly_momentum <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_momentum) <- names(monthly_returns)
# Iterate over rows and compute momentum.
for (i in 2:nrow(monthly_momentum)) {
    current_day <- index(monthly_momentum[i])
    end_day <- index(monthly_momentum[i - 1])
    start_day <- end_day %m-% months(10)
    dates <- paste0(start_day, '/', end_day)
    returns <- monthly_returns[dates]
    if (nrow(returns) < 11) { next }
    for (j in 1:ncol(monthly_momentum)) {
        monthly_momentum[current_day, j] <- 100 * (prod(returns[, j] + 1) - 1)
    }
}
tail(monthly_momentum,3)[, 1:5]

                   F      CSX       OKE        DFS      ITW
2020-10-31 -21.74476 11.27453 -59.63724 -26.343283 16.77008
2020-11-30 -13.79337 10.83510 -54.86384 -21.957981 14.60072
2020-12-31  -1.22478 25.51735 -47.51509  -7.860505 19.29762

### <span style='background: lightblue'>1.8. Summary Statistics</span>

* **Create Function for Statistics**

In [17]:
getStat <- function (variable_data) {
    variable_data <- apply(variable_data, 2, function(x) ifelse(is.finite(x), x, NA))
    df <- na.omit(variable_data) 
    stats <- apply(df, 1,function(y){ c(mean(y), sd(y), skewness(y),
        kurtosis(y),min(y), quantile(y, probs = .05, na.rm = TRUE),
        quantile(y, probs = .25, na.rm = TRUE), median(y), 
        quantile(y, probs = .75, , na.rm = TRUE),
        quantile(y, probs = .95, na.rm = TRUE), max(y))
    })
    return (rowMeans(stats))    
}
                           
getStat1 <- function (variable_data) {
    variable_data <- apply(variable_data, 2, function(x) ifelse(is.na(x), 0,x))
    df <- na.omit(variable_data) 
    stats <- apply(df, 1,function(y){ c(mean(y), sd(y), skewness(y),
        kurtosis(y),min(y), quantile(y, probs = .05, na.rm = TRUE),
        quantile(y, probs = .25, na.rm = TRUE), median(y), 
        quantile(y, probs = .75, , na.rm = TRUE),
        quantile(y, probs = .95, na.rm = TRUE), max(y))
    })
    stats <- apply(stats, 2, function(x) ifelse(is.na(x), 0,x))                       
    return (rowMeans(stats))    
}                          

* **Show Factor Summary Statistics**

In [18]:
#name the row: mean, variance, skewness, excess kurtosis, minimum and maximum
col_n <- c('Mean','SD', 'Skew', 'Kurt', 'Min','5%',
        '25%', 'Median', '75%', '95%','Max', 'n')
row_n <- c('Beta','MktCap', 'Size', 'Value', 'Mom','Illiq', 'Return')
stock_n <- c(250,250,250,250,250,250,250)
df_stat <- data.frame(getStat(monthly_betas))
df_stat <- cbind(df_stat, getStat(monthly_market_caps)) 
df_stat <- cbind(df_stat, getStat(monthly_sizes)) 
df_stat <- cbind(df_stat, getStat1(monthly_bm_ratios)) 
df_stat <- cbind(df_stat, getStat(monthly_momentum)) 
df_stat <- cbind(df_stat, getStat(monthly_illiq)) 
df_stat <- cbind(df_stat, getStat(monthly_returns)) 
stats_t <- as.data.frame(t(df_stat))
stats_t <- cbind(stats_t, stock_n) 
out_res <- data.frame(lapply(stats_t, function(y) if(is.numeric(y)) round(y, 3) else y))    
rownames(out_res) <- row_n
colnames(out_res) <- col_n
out_res

,Mean,SD,Skew,Kurt,Min,5%,25%,Median,75%,95%,Max,n
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Beta,1.007,0.331,0.241,3.596,0.215,0.464,0.795,0.999,1.215,1.550,2.107,250
MktCap,62.301,143.790,6.834,59.354,3.631,6.836,11.993,24.318,53.839,219.215,1518.211,250
Size,3.324,1.104,0.839,3.591,1.270,1.913,2.477,3.186,3.981,5.387,7.320,250
Value,0.426,0.460,1.027,6.376,-1.170,0.000,0.108,0.318,0.630,1.255,2.707,250
Mom,14.437,23.467,1.049,8.344,-48.015,-18.650,-0.270,12.848,26.677,51.919,137.387,250
Illiq,1.618,2.366,3.636,27.254,0.054,0.137,0.553,1.018,1.911,4.443,24.706,250
Return,0.014,0.063,0.301,6.243,-0.204,-0.082,-0.022,0.014,0.050,0.113,0.283,250


# 2. Univariate portfolio sorts and analysis

Your analysis will start with univariate portfolio sort. For each sort variable using monthly rebalancing sort the stocks into the quintiles, i.e. make 5 portfolios, and create equally weighted and value weighted portfolios. 

 

For each sort variable and each quintile report average values of sorting variable (similar to Lecture 10 - Momentum, slide 9, Panel A). Further, for each sort variable, each quintile and difference “5-1“ portfolio report time-series averages with corresponding Newey-West t-stats of one-month-ahead excess return and Fama-French-Carhart-Pastor-Stambaugh alpha. 

* **Create Univariate Portfolio Sorting Functions**

get_alpha() function calculates FFCPS alpha by using downloaded factors from the web of Kenneth French and Lubos Pastor

In [19]:
# use 1 as equal_weights input parameters
monthly_equal_weights <- monthly_betas
monthly_equal_weights[,] <- 1

get_alpha <- function (port_return) {
    names(port_return) <- "return"
    sample <- cbind(port_return, factor_data)    
    # FFCPS model
    model_FFCPS <- lm('return ~ liq + mkt.rf + smb + hml + mom ', data = sample)     
    return(model_FFCPS)    
}

univariate_sort() function calculate univariate sort, and for each sorted group output: average factor value, excess returns, NeweyWest t-stats, FFCPS alpha, alpha t-value.

In [20]:
# Parameters: factor variable want to be sorted, stock returns, 
# market_caps（weight） for VW or EV
# n as number for portfolios of each sorted variables
univariate_sort <- function (sort_variable_data, returns, market_caps,
        n, factor_name, round_n = 6) {
    row_n <- c(factor_name , 'Return',' t-stats', 'Alpha',' t-value' )
    diff_portfolio <- paste(n, '- 1')
    # Initialize an xts object for storing portfolio returns.
    portfolio_returns <- as.xts(
        matrix(
            nrow = nrow(sort_variable_data) - 1,
            ncol = n + 1
        ),
        order.by = index(sort_variable_data[2:nrow(sort_variable_data)])
    )
    names(portfolio_returns) <- c(1:n, diff_portfolio)        
    # Initialize an xts object for storing portfolio factors
    # to output avarge beta after grouping
    portfolio_factors <- as.xts(
        matrix(
            nrow = nrow(sort_variable_data) - 1,
            ncol = n 
        ),
        order.by = index(sort_variable_data[2:nrow(sort_variable_data)])
    )
    names(portfolio_factors) <- c(1:n)            
    # Iterate over rows, find breakpoints and compute monthly 
    # returns within the given value breakpoints.
    # month_factor
    for (i in 1:nrow(portfolio_returns)) {
        current_month <- index(portfolio_returns[i])
        # use the forward looking result, so need to get next month return and markcap
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), 
                    frac = 1)
        if (!next_month %in% index(returns)) {
            next
        }
        # Avoid look-ahead bias by sorting the returns
        # after the month used to compute breakpoints ends.
        # use this month sort variable, and next month return and markcap
        # because need to use the forward month return for result
        month_returns <- returns[next_month]        
        # just use month_factor as input sort variable parameter
        month_factor <- sort_variable_data[current_month]        
        month_market_caps <- market_caps[next_month]
        # Replace NA market caps with 0s so that such observations have no weights.
        # use markcap for VW
        month_market_caps[is.na(month_market_caps)] <- 0        
        # input parameter n is break group numbers
        # set up breakpoint value based on sort variables
        # for each month, it has its own breakpoints
        breakpoints <- quantile(month_factor, 0:n/n, na.rm = TRUE)
        not_na <- !is.na(month_factor)   # removed not traded data        
        # j is for each portfolio group
        for (j in 1:n) {
            # compare sort varible with breakpoint
            # filter is a boolean type flag
            # if in this group, filter is True
            # decide the stock is included in this portfolio or not
            filter <- (breakpoints[[j]] < month_factor) & (
                month_factor < breakpoints[[j + 1]]) & not_na            
            # Compute weighted average portfolio returns within the groups
            # use month_market_caps parameter for weightings
            # for EV, input is 1
            # use the returns of next months for calculation
            # only stocks in the portfolio (filter = True) are included for calculation
            portfolio_returns[i, j] <- weighted.mean(t(month_returns[, filter]), 
                t(month_market_caps[, filter]))                      
            # calculate avarge beta of each group of each month
            portfolio_factors[i, j] <- mean(month_factor[, filter])        
        }
        # calculate diff of the avg returns between last and first group
        portfolio_returns[i, diff_portfolio] <- portfolio_returns[i, 
                n] - portfolio_returns[i, 1]
    }      
    # Now in portfolio_returns, we have the avg returns of each month 
    # in each group of portfolios and the diff group
    # Compute overall average returns within portfolios and their standard errors.
    # just 2 rows, n+1 portfolios, the last one is diff portfolio    
    # Create empty frame  results_returns
    results_returns <- as.data.frame(matrix(nrow = 5, ncol = n + 1))
    names(results_returns) <- c(1:n, diff_portfolio)
    # calculate avarge beta in all months for sub-group    
    mean_factor <- colMeans(na.omit(portfolio_factors))       
    for (j in 1:n ) {
        results_returns[1, j] <- mean_factor[j]
    }    
    results_returns[1, n+1] <- mean_factor[n] - mean_factor[1]            
    # every i is each stock
    for (i in 1:ncol(results_returns)) {
        # run simple regression of all months for each stock
        model <- lm(na.omit(portfolio_returns[, i]) ~ 1)
        # contains alpha
        results_returns[2, i] <- model$coefficients[[1]]
        # alpha / standard errors
        results_returns[3, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, 
            lag = 6))[[1]]        
        # calculate FFCPS alpha and t-value 
        model <- get_alpha(portfolio_returns[, i])        
        results_returns[4, i] <- summary(model)$coefficients[1, 1]
        results_returns[5, i] <- summary(model)$coefficients[1, 3]        
    }            
    # round numbers
    out_res <- data.frame(lapply(results_returns, function(y) if(
        is.numeric(y)) round(y, round_n) else y))  
    rownames(out_res) <- row_n
    colnames(out_res) <- c(1:n, diff_portfolio)                                 
    out_res                                                                 
}

perform_fama_macbeth_regression() funtion is from the seminar, it calculates stock level factor regression.

In [21]:
#  data_explained are monthly returns, data_explanatory are factors
perform_fama_macbeth_regression <- function (data_explained, 
            data_explanatory) {    
    # Initialize an xts object for storing regression results.
    fm_results <- as.xts(
        matrix(
            # every row is each month
            nrow = nrow(data_explained) - 1,
            # every column is each factor
            ncol = 1 + length(data_explanatory)
        ),
        # start from the second month to last month
        order.by = index(data_explained[2:nrow(data_explained)])
    )
    column_names <- c('Intercept', names(data_explanatory))
    names(fm_results) <- column_names
    # Iterate over rows and perform cross-sectional regressions.
    # each i is for each month
    for (i in 1:nrow(fm_results)) {
        current_month <- index(fm_results[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), 
                frac = 1)
        if (!next_month %in% index(data_explained)) {
            next
        }
        # Avoid look-ahead bias by sorting the returns 
        # after the month used to compute breakpoints ends.        
        # load in return data as the t+1 month
        month_data_explained <- data_explained[next_month]
        month_data_explanatory <- c()        
        # each j is one factor
        for (j in 1:length(data_explanatory)) {            
            if (!current_month %in% index(data_explanatory[[j]])) {
                move_to_next_month <- TRUE
                break
            }
            move_to_next_month <- ncol(data_explanatory[[j]][current_month]) == sum(
                is.na(data_explanatory[[j]][current_month]))            
            if (move_to_next_month) {
                break
            }            
            cross_section <- t(data_explanatory[[j]][current_month])
            colnames(cross_section) <- names(data_explanatory)[j]
            cross_section[which(cross_section == -Inf)] = NA
            cross_section[which(cross_section == Inf)] = NA
            month_data_explanatory <- cbind(month_data_explanatory, cross_section)
        } # end of j
        if (move_to_next_month) {
            next
        }
        transposed_explained <- t(month_data_explained)
        colnames(transposed_explained) <- c('explained')
        # create formula for linear regression
        equation <- paste('explained ~ 1 + ', 
            paste0(colnames(month_data_explanatory), collapse = ' + '))
        # create data frame of all factors
        all_data <- na.omit(as.data.frame(cbind(transposed_explained, 
            month_data_explanatory)))
        # Save regression coefficients and other statistics.
        model <- lm(equation, data = all_data)
        # save the result for each factor
        for (k in 1:ncol(fm_results)) {
            fm_results[i, k] <- model$coefficients[[k]]
        }
    } # end of i 
    # Compute time series means, standard errors.
    final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = ncol(fm_results)))
    names(final_fm_results) <- colnames(fm_results)
    # every i is each factor
    for (i in 1:ncol(final_fm_results)) {
        # run regression for all months of each factor
        model <- lm(na.omit(fm_results[, i]) ~ 1)
        final_fm_results[1, i] <- model$coefficients[[1]]
        final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, 
            lag = 4))[[1]]
    }    
    output <- round(final_fm_results, digits = 5)
    output[is.na(output)] <- ""   
    output
}

### <span style='background: lightblue'>2.1. Portfolio sorts and analysis based on beta</span>

**Sort the stocks into the quintiles**

**2.1.1. Create equal-weighted portfolios**

In [22]:
univariate_sort(monthly_betas, monthly_returns, monthly_equal_weights, 
        5, "Beta", 4)

,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Beta,0.5649,0.8377,1.0051,1.1855,1.5188,0.9539
Return,0.0092,0.0114,0.0113,0.0141,0.0139,0.0047
t-stats,3.2582,3.0375,2.7310,2.7870,2.1523,0.9914
Alpha,0.0089,0.0103,0.0108,0.0131,0.0129,0.0040
t-value,2.9604,2.8112,2.5112,2.5339,1.9942,0.8146


**Comments:**

First we exam the difference portfolio. The average return of the difference portfolio, presented in the column labeled 5-1, is 0.46%. This difference is not statistically distinguishable from zero as the t-statistic is 0.99, which is smaller than 2. Thus, our portfolio analysis fails to detect a cross-sectional relation between 𝛽 and excess stock returns.

In terms of the portfolio 1 to 5, the results are 0.92%, 1.14%, 1.13%, 1.41%,and 1.39%, respectively.Each of these average returns is found to be statistically significant as all the t-statistics are larger than 2.This indicates that in the average year, each of these seven portfolios produces positive excess returns. This is not surprising because stocks are known to generate average returns that are higher than the return on the risk-free security. However,when comparing the average returns of portfolio 2 (1.14%) and 3(1.13%), as well as those of portfolio 4(1.41%) and portfolio 5(1.39%), we found that portfolio with higher betas do not generate higher average returns.

**2.1.2. Create value-weighted portfolios**

In [23]:
univariate_sort(monthly_betas, monthly_returns, monthly_market_caps, 
            5, "Beta", 4)

,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Beta,0.5649,0.8377,1.0051,1.1855,1.5188,0.9539
Return,0.0100,0.0118,0.0118,0.0139,0.0155,0.0055
t-stats,4.0252,3.4321,2.9321,3.0953,2.6196,1.1411
Alpha,0.0099,0.0108,0.0115,0.0134,0.0153,0.0054
t-value,3.4223,2.9010,2.7244,2.7831,2.4788,1.0816


**Comments:**

Similar to the equal-weighted portfolios, the cross-sectional relation between 𝛽 and excess stock returns does not exist for value-weighted portfolios. Because the t-statistic for the difference portfolio is 1.14, which is smaller than 2. 

When examing the individual decile portfolio returns, we found all portfolios generate statistically significant positive average returns, since all t-statistics are larger than 2.The average returns for portfolio 1 to 5 are 1.00%, 1.18%,1.18%, 1.39%, and 1.55%, respectively. The reason is that stocks generate average returns that are higher than the return on the risk-free security.Meanwhile, in the value-weighted portfolios, portfolios with higher beta usually generate higher average returns than portfolios with lower betas. 

**2.2.3. Fama-MacBeth regression analysis**

In [24]:
data <- list(monthly_betas)
names(data) <- c('Beta')
perform_fama_macbeth_regression(monthly_returns, data)

Intercept,Beta
<dbl>,<dbl>
0.00655,0.0047
1.75325,0.9234


**Comments:**

In the Fama-Macbeth regression, beta is the only independent variable and the average slope from the monthly cross-sectional regressions is 0.0047.But the t-statistic for it is 0.923, wich is smaller than 2. Thereofore, the relation between betas and future stock returns  is statistically insignificant.

### <span style='background: lightblue'>2.2. Portfolio sorts and analysis based on size</span>

**Sort the stocks into the quintiles**

**2.2.1. Create equal-weighted portfolios**

In [25]:
univariate_sort(monthly_sizes, monthly_returns,  monthly_equal_weights, 
            5, "Size", 4)

,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Size,1.4852,2.0779,2.6229,3.2802,4.4557,2.9705
Return,0.0203,0.0107,0.0098,0.0103,0.0079,-0.0124
t-stats,4.2525,2.6175,2.1152,2.6703,2.1880,-4.3873
Alpha,0.0198,0.0100,0.0086,0.0095,0.0073,-0.0125
t-value,3.8172,2.3975,2.0489,2.4407,1.9597,-4.8301


**Comments:**

Focusing first on the difference portfolio, which is long large stocks (the decile 5 portfolio) and short small stocks (the decile one portfolio), the results indicate that this portfolio generates an economically large average return of −1.23%, which is highly statistically signifcant with a Newey and West t-statistic of −4.38. In other words,we can conclude from the column "5-1" that there is an economically important and statistically signifcant relation between Size and future stock returns, and such relationship is negative. 

Examining the individual quintile portfolio returns, we see that quintile portfolio one generates a substantially higher average excess return  than any of the other quintile portfolios.There is also a large drop in performance between quintile portfolios one and two. For portfolios sorted on Size, the frst quintile portfolio generates an average monthly excess return of 2.03%， while the second quintile portfolio only generates an average monthly excess return of 1.07%. The average excess returns of portfolios 2 through 5 are relatively similar.All five portfolios has statistically sinificant Newey and West t-statistic. Therefore, we can find that the cross-sectional relation between Size and future stock returns appears to be driven primarily by the high abnormal returns of the smallest stocks in the sample. The result supports the size effect that stocks with large market capitalizations (large size) tend to have lower returns than stocks with small market capitalizations (small size).

**2.2.2. Create value-weighted portfolios**

In [26]:
univariate_sort(monthly_sizes, monthly_returns,  monthly_market_caps, 
            5, "Size", 4)

,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Size,1.4852,2.0779,2.6229,3.2802,4.4557,2.9705
Return,0.0261,0.0159,0.0153,0.0145,0.0113,-0.0148
t-stats,5.1057,4.0884,3.4670,3.9894,3.5743,-4.0585
Alpha,0.0260,0.0156,0.0144,0.0141,0.0109,-0.0151
t-value,4.9347,3.7754,3.5224,3.7058,3.0275,-5.0620


**Comments:**

When value-weighted portfolios are used, the negative relationship between Size and future stock returns is also detected.The average return of the Size-sorted difference portfolio of -1.48% (t-statistic = -4.058)is statistically significant. In addition, similarlo to the equal-weighted porfolio,the quintile portfolio one of value-weighted portfolios generates a substantially higher average excess return than any of the other quintile portfolios.

However, we could also find that the magnitudes of the average return of the value-weighted portfolios are larger than the average return of the equal weighted portfolios in every quintile portfolio.Focusing on the effect of value-weighting on the returns of the first quintile portfolio, we can acknowledge that the first quintile portfolio already contains only stocks with low market capitalizations. As the average excess return of value-weighted quintile portfolio one(2.6%) is larger than that of equal-weighted quintile portfolio one(2%), the result therefore,indicates that the stocks with the lowest market capitalizations have extremely high returns and are thus driving the value-weighted portfolio results.   

结论？？？

**2.2.3. Fama-MacBeth regression analysis**

In [27]:
data <- list(monthly_sizes)
names(data) <- c('Size')
perform_fama_macbeth_regression(monthly_returns, data)

Intercept,Size
<dbl>,<dbl>
0.02008,-0.00319
3.84687,-4.91179


**Comments:**

In this FM regression analyses, we take size as the only independent variable in the regression specification,and the average slope from the monthly cross-sectional regressions is -0.00319 with a cirresponding NW t-statistic of -4.9117, indicating a statistically significant negative relation between Size and future stock returns. 

### <span style='background: lightblue'>2.3. Portfolio sorts and analysis based on liquidity</span>

**Sort the stocks into the quintiles**

**2.3.1. Create equal-weighted portfolios**

In [28]:
# call for Equally weighted sorting first
univariate_sort(monthly_illiq, monthly_returns, monthly_equal_weights,
            5, "Illiq", 4)

,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Illiq,0.3175,0.8385,1.4781,2.5704,6.8691,6.5516
Return,0.0094,0.0098,0.0098,0.0133,0.0170,0.0076
t-stats,2.5710,2.3788,2.1626,3.1959,3.7993,3.8426
Alpha,0.0087,0.0091,0.0084,0.0126,0.0165,0.0078
t-value,2.3452,2.2337,1.9631,2.8897,3.4686,3.7609


**Comments:**   

For equal-weighted portfolios formed by sorting on ILLIQ,the average excess return increases from 0.93% per month for the first
quintile portfolio to 1.69% per month for the 5th quintile portfolio.All portfolios has statistically significant NW t-statistics, meaning that the average excess returns of all portfolios are statistically different from zero. The average return of the equal-weighted ILLIQ 5-1 portfolio of 0.76% per month is economically large and statistically signifcant, with a NW t-statistic of 3.84. As high value of ILLIQ factor portfolio has low liquid, portfolio 5 is the most illiquid portfolio, and it has the highest returns. We could therefore conclude that as ILLIQ increase(liquidity decrease), portfolio returns increase. So ILLIQ factor is positively related with stock returns.

**2.3.2. Create value-weighted portfolios**

In [29]:
univariate_sort(monthly_illiq, monthly_returns, monthly_market_caps, 
         5, "Illiq", 4)
# high value lower liquid, high returns, positive effect

,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Illiq,0.3175,0.8385,1.4781,2.5704,6.8691,6.5516
Return,0.0118,0.0127,0.0139,0.0173,0.0192,0.0075
t-stats,3.6440,3.4185,3.2171,4.0212,4.5487,3.7515
Alpha,0.0113,0.0121,0.0129,0.0169,0.0188,0.0076
t-value,3.1458,3.1398,3.1315,3.9360,4.2031,3.6496


**Comments:**

The results of the value-weighted portfolio analyses is quite similar to what we have found in equal-weighted portfolio analyses. A positive relation between ILLIQ and future stock returns is also detected in the value-weighted portfolio analyses.From quintile portfolio 1 to 5, monthly returns increase from 1.17% to 1.92%. The average return of the value-weighted ILLIQ 5-1 portfolio of 0.74% per month is economically large and statistically signifcant, with a NW t-statistic of 3.75.

**2.3.3. Fama-MacBeth regression analysis**

In [30]:
data <- list(monthly_illiq)
names(data) <- c('Illiquidity')
perform_fama_macbeth_regression(monthly_returns, data)
# has expected direction

Intercept,Illiquidity
<dbl>,<dbl>
0.00926,0.00163
2.29265,3.52832


**Comments:**

In this FM regression analyses, we take illquidity factor as the only independent variable in the regression specification,and the average slope from the monthly cross-sectional regressions is 16.27 with a corresponding NW t-statistic of 3.52, indicating a statistically significant and positive relation between illiquidity and future stock returns.This result is consistent with what we have found in univariate portfolio analyses.

# 3. Bivariate portfolio analysis

In the bivariate portfolio analysis you will use independent sorting procedure and study only value-weighted portfolios. Given the relatively limited amount of stocks you are analysing, create 3x3 portfolios - use the 30 and 70 percentiles of the sort variables as breakpoints. Report the results for the 
* Size-Beta sort
* Size-Third_factor sort, i.e. Size and the same sort variable as in univariate sort. 

Similar to univariate portfolio analysis, for each sort variable, each percentiles group and difference "3-1" portfolio report time-series averages with corresponding Newey-West t-stats of one-month-ahead/one-quarter-ahead excess return and Fama-French-Carhart-Pastor-Stambaugh alpha (similar to Lecture 10 - Value, slide 15).

* **Create Bivariat Portfolio Sorting Function**

In [31]:
# Parameters: factor variable want to be sorted, stock returns, 
# market_caps（weight） for VW or EV, EV just set market_caps as 1
# only for 3X3 sorting, expandable later for m X n
bivariat_sort <- function (sort_data1, sort_data2, returns, 
        market_caps, round_n= 6, m=3, n=3) {
    # use beta and size to do independent sorting
    # 3 groups for beta and 3 groups for size
    n_portfolios_beta <- m
    n_portfolios_size <- n
    n_portfolios <- n_portfolios_beta * n_portfolios_size     
    diff_portfolios <- c('3 - 1', ' 6 - 4', '9 - 7', '7 - 1', ' 8 - 2', '9 - 3')    
    dimC <- c(0, 0.3, 0.7, 1)      
    # Initialize an xts object for storing portfolio returns.
    returns_sorts <- as.xts(
        matrix(
            nrow = nrow(sort_data1) - 1,
            ncol = n_portfolios + m + n
        ),
        order.by = index(sort_data1[2:nrow(sort_data1)])
    )
    row_n <- c( 'Return',' t-stats', 'Alpha',' t-value' )   
    col_n <- c(1:n_portfolios, diff_portfolios)
    names(returns_sorts) <- col_n            
    # Iterate over rows, find breakpoints and compute monthly returns 
    # within the given beta and size breakpoints.
    for (i in 1:nrow(returns_sorts)) {
        current_month <- index(returns_sorts[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), 
            frac = 1)
        if (!next_month %in% index(returns)) { next }
        # Avoid look-ahead bias by sorting the returns after the month 
        # used to compute breakpoints ends.
        # use this month's beta and size, next month's return and market cap(VW)
        month_returns <- returns[next_month]
        month_betas <- sort_data1[current_month]
        month_sizes <- sort_data2[current_month]
        month_market_caps <- market_caps[next_month]
        # Replace NA market caps with 0s so that such observations have no weights.        
        month_market_caps[is.na(month_market_caps)] <- 0
        if (ncol(month_betas) == sum(is.na(month_betas))) { next }
        # break point for beta groups
        breakpoints_betas <- quantile(month_betas,
            dimC, na.rm = TRUE)
        not_na_betas <- !is.na(month_betas)
        # first sort beta
        for (j in 1:n_portfolios_beta) {
            filter_betas <- (breakpoints_betas[[j]] < month_betas) & (
                month_betas < breakpoints_betas[[j + 1]]) & not_na_betas
            # catch first factor sorting results
            filtered_month_returns <- month_returns[, filter_betas]
            filtered_month_sizes <- month_sizes[, filter_betas]
            filtered_month_market_caps <- month_market_caps[, filter_betas]
            # only difference of dependent and independent sorting is
            # how to calculate breakpoint:
            # use filtered_month_sizes for dependent sorting
            # use month_sizes for independent sorting            
            breakpoints_sizes <- quantile(month_sizes, dimC, na.rm = TRUE)        
            not_na_sizes <- !is.na(month_sizes)
            # second sort size after sorted beta
            # for independent, the breakpoint is whole dataset
            for (k in 1:n_portfolios_size) {
                filter_sizes <- (breakpoints_sizes[[k]] < month_sizes) & (
                    month_sizes < breakpoints_sizes[[k + 1]]) & not_na_sizes
                # Compute weighted average portfolio returns.
                # match the columns(only for independent)
                filter_sizes <- filter_sizes[,colnames(filtered_month_returns)]
                returns_sorts[i, k + (j - 1) * n_portfolios_size] <- weighted.mean(t(
                    filtered_month_returns[, filter_sizes]), 
                    t(filtered_month_market_caps[, filter_sizes]))            
            }
            # calculate diff of factor 1
            returns_sorts[i, diff_portfolios[j]] <- returns_sorts[i, 
                    n_portfolios_size * j] - returns_sorts[i, 
                    n_portfolios_size * (j - 1) + 1]
        }        
        # calculate diff of factor 2 
        for ( t in 1:n ) {
            returns_sorts[i, diff_portfolios[m+t]] <- returns_sorts[i, 
                    n * (m - 1) + t] - returns_sorts[i,t]
        }
    }    
    # Compute overall average returns within portfolios and their standard errors.
    results_xts <- as.data.frame(matrix(nrow = 4, 
        ncol = n_portfolios + m + n ))
    for (i in 1:ncol(results_xts)) {
        model <- lm(na.omit(returns_sorts[, i]) ~ 1)
        results_xts[1, i] <- model$coefficients[[1]]
        results_xts[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, 
            lag = 6))[[1]]
        # calculate FFCPS alpha
        model <- get_alpha(returns_sorts[, i])        
        results_xts[3, i] <- summary(model)$coefficients[1, 1]
        results_xts[4, i] <- summary(model)$coefficients[1, 3]             
    }
    # round numbers
    out_res <- data.frame(lapply(results_xts, function(y) if(
        is.numeric(y)) round(y, round_n) else y))  
    rownames(out_res) <- row_n
    colnames(out_res) <- col_n                                                                      
    return(out_res)
}

In [32]:
# Create Function for Result Output Table
getName <- function(factor_in, n = 3){   
    row_s <- c()
    for (i in 1:n ) {
        row_name <- paste(factor_in, i)
        row_s <- cbind(row_s, row_name)
    }
    row_name <- paste(factor_in, "Avg")
    row_s <- cbind(row_s, row_name)    
    row_name <- paste(factor_in, "3-1")
    row_s <- cbind(row_s, row_name)        
    return (row_s)
}

format_result <- function(res_in, fact1, fact2, 
        round_n= 6, multi = 1, n = 3){       
    # Create data frame.
    col_t <- fact1
    row_t <- fact2 
    col_name <-  getName(col_t)
    row_name <-  getName(row_t)
    # all return is percentage, multiple by 100
    res_100 <- res_in * multi
    # Create the data frame.
    t_size <- n 
    df_t <- data.frame(matrix(ncol = t_size, nrow = t_size))
    # for each column
    for (j in 1:n ) {
        # for each row within the column
        for (i in 1:n ) {
            df_t[i, j] <- res_100[(j-1) * n + i]        
        }        
    }    
    # get mean
    df_t <- rbind(df_t, colMeans(df_t)) 
    df_t <- cbind(df_t, rowMeans(df_t))
    df_t <- round(df_t, round_n)
    # get diff
    diff_r = c() # for mean
    diff_c = c()
    # for row
    for (j in 1: n ) {
        df_t[5, j] <- round(res_100[(n * n + j)],round_n)
        diff_r[j] <- df_t[5, j]
    }
    # for column
    for (i in 1: n ) {
        df_t[i, 5] <- round(res_100[(n * (n+1) + i)],round_n)  
        diff_c[i] <- df_t[i, 5]
    }
    # clean format
    df_t[4, 4] <- ""
    df_t[4, 5] <- round(mean(diff_c),round_n)
    df_t[5, 4] <- round(mean(diff_r),round_n) 
    df_t[is.na(df_t)] <- ""   
    colnames(df_t) <- col_name
    rownames(df_t) <- row_name
    df_t
}

### <span style='background: lightblue'>3.1. Size-Beta Sort</span>

* **Size-Beta Independent value-weighted(WV) sorting**  


In [33]:
# input parameters: sort factor1, sort factor2, excess returns
# mktCap for VW, rounding
res_SizeBeta <- bivariat_sort(monthly_sizes, monthly_betas,  
        monthly_returns, monthly_market_caps, 5)
res_SizeBeta
# high value lower liquid, high returns, positive effect

,1,2,3,4,5,6,7,8,9,3 - 1,6 - 4,9 - 7,7 - 1,8 - 2,9 - 3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Return,0.01838,0.02025,0.02577,0.01120,0.01541,0.01748,0.00986,0.01141,0.01400,0.00738,0.00629,0.00415,-0.00853,-0.00883,-0.01177
t-stats,5.71209,4.60778,4.07621,3.70669,3.66559,2.84180,4.15581,3.00891,2.59152,1.75321,1.48381,0.97819,-4.18314,-4.62122,-3.30688
Alpha,0.01827,0.01958,0.02491,0.01073,0.01461,0.01670,0.00971,0.01094,0.01447,0.00664,0.00597,0.00476,-0.00856,-0.00864,-0.01044
t-value,5.27474,3.83783,3.81553,3.53672,3.47301,2.76328,3.26766,2.66061,2.46164,1.45488,1.35024,1.09127,-3.82864,-3.40265,-2.91968


**Size Small: 1, 2, 3;  Size Mid: 4, 5, 6; Size Big: 7, 8, 9  
Beta low: 1, 4, 7; Beta Mid: 2, 5, 8; Beta High: 3, 6, 9**  


**Output excess return result**

In [34]:
# input parameters: result from bivariat_sort, 
# sort factor1 name, sort factor2 name, rounding, multipler
format_result(res_SizeBeta, "Size", "Beta", 5)

,Size 1,Size 2,Size 3,Size Avg,Size 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Beta 1,0.01838,0.01120,0.00986,0.01315,-0.00853
Beta 2,0.02025,0.01541,0.01141,0.01569,-0.00883
Beta 3,0.02577,0.01748,0.01400,0.01908,-0.01177
Beta Avg,0.02147,0.01470,0.01176,,-0.00971
Beta 3-1,0.00738,0.00629,0.00415,0.00594,


**Output result by percentage(multiple by 100)**

In [35]:
format_result(res_SizeBeta, "Size", "Beta", 3, 100)

,Size 1,Size 2,Size 3,Size Avg,Size 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Beta 1,1.838,1.120,0.986,1.315,-0.853
Beta 2,2.025,1.541,1.141,1.569,-0.883
Beta 3,2.577,1.748,1.400,1.908,-1.177
Beta Avg,2.147,1.470,1.176,,-0.971
Beta 3-1,0.738,0.629,0.415,0.594,


**Output result of t-status**

In [36]:
format_result(res_SizeBeta[2,], "Size", "Beta", 3)

,Size 1,Size 2,Size 3,Size Avg,Size 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Beta 1,5.712,3.707,4.156,4.525,-4.183
Beta 2,4.608,3.666,3.009,3.761,-4.621
Beta 3,4.076,2.842,2.592,3.17,-3.307
Beta Avg,4.799,3.405,3.252,,-4.037
Beta 3-1,1.753,1.484,0.978,1.405,


**Output result of FFCPS Alpha**

In [37]:
format_result(res_SizeBeta[3,], "Size", "Beta", 5)

,Size 1,Size 2,Size 3,Size Avg,Size 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Beta 1,0.01827,0.01073,0.00971,0.0129,-0.00856
Beta 2,0.01958,0.01461,0.01094,0.01504,-0.00864
Beta 3,0.02491,0.01670,0.01447,0.01869,-0.01044
Beta Avg,0.02092,0.01401,0.01171,,-0.00921
Beta 3-1,0.00664,0.00597,0.00476,0.00579,


**Output result of Alpha t-value**

In [38]:
format_result(res_SizeBeta[4,], "Size", "Beta", 5)

,Size 1,Size 2,Size 3,Size Avg,Size 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Beta 1,5.27474,3.53672,3.26766,4.02637,-3.82864
Beta 2,3.83783,3.47301,2.66061,3.32382,-3.40265
Beta 3,3.81553,2.76328,2.46164,3.01348,-2.91968
Beta Avg,4.30937,3.25767,2.79664,,-3.38366
Beta 3-1,1.45488,1.35024,1.09127,1.2988,


* **Beta-Size sort to verify results**

In [39]:
res_BetaSize <- bivariat_sort(monthly_betas, monthly_sizes, 
        monthly_returns, monthly_market_caps, 5)
res_BetaSize 
# high value lower liquid, high returns, positive effect

format_result(res_BetaSize, "Beta", "Size", 5)

,1,2,3,4,5,6,7,8,9,3 - 1,6 - 4,9 - 7,7 - 1,8 - 2,9 - 3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Return,0.01838,0.01120,0.00986,0.02025,0.01541,0.01141,0.02577,0.01748,0.01400,-0.00853,-0.00883,-0.01177,0.00738,0.00629,0.00415
t-stats,5.71209,3.70669,4.15581,4.60778,3.66559,3.00891,4.07621,2.84180,2.59152,-4.18314,-4.62122,-3.30688,1.75321,1.48381,0.97819
Alpha,0.01827,0.01073,0.00971,0.01958,0.01461,0.01094,0.02491,0.01670,0.01447,-0.00856,-0.00864,-0.01044,0.00664,0.00597,0.00476
t-value,5.27474,3.53672,3.26766,3.83783,3.47301,2.66061,3.81553,2.76328,2.46164,-3.82864,-3.40265,-2.91968,1.45488,1.35024,1.09127


,Beta 1,Beta 2,Beta 3,Beta Avg,Beta 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Size 1,0.01838,0.02025,0.02577,0.02147,0.00738
Size 2,0.01120,0.01541,0.01748,0.0147,0.00629
Size 3,0.00986,0.01141,0.01400,0.01176,0.00415
Size Avg,0.01315,0.01569,0.01908,,0.00594
Size 3-1,-0.00853,-0.00883,-0.01177,-0.00971,


**Beta low: 1, 2, 3; Beta Mid: 4, 5, 6; Beta High: 7, 8, 9  
Size Small: 1, 4, 7;  Size Mid: 2, 5, 8; Size Big: 3, 6, 9**

* **Fama-MacBeth regressions of returns on size and beta**

In [40]:
data <- list(monthly_sizes, monthly_betas)
names(data) <- c('Size', 'Beta' )
perform_fama_macbeth_regression(monthly_returns, data)

Intercept,Size,Beta
<dbl>,<dbl>,<dbl>
0.01497,-0.00280,0.00339
3.73317,-4.63646,0.67016


### <span style='background: lightblue'>3.2. Size-Illiq sort</span>

* **Size-Illiq Independent value-weighted(WV) sorting**

In [41]:
res_SizeIlliq <- bivariat_sort( monthly_sizes, monthly_illiq, 
    monthly_returns, monthly_market_caps, 5)

print("Original Result")
res_SizeIlliq
# high value lower liquid, high returns, positive effect
print("Excess Return %")
format_result(res_SizeIlliq, "Size", "Illiq", 2, 100)
print("t-stats of Return")
format_result(res_SizeIlliq[2,], "Size", "Illiq", 3)
print("FFCPS Alpha %")
format_result(res_SizeIlliq[3,], "Size", "Illiq", 2, 100)

[1] "Original Result"


,1,2,3,4,5,6,7,8,9,3 - 1,6 - 4,9 - 7,7 - 1,8 - 2,9 - 3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Return,0.11546,0.02653,0.02050,0.01710,0.01418,0.01382,0.01139,0.00986,0.06332,-0.07431,-0.00327,0.07903,-0.08885,-0.01667,0.07969
t-stats,2.18410,5.59127,4.66730,4.42621,3.41464,3.32204,3.58964,2.40753,4.55880,-1.59097,-1.50303,6.95944,-1.89592,-5.56181,7.57897
Alpha,0.18910,0.02589,0.02024,0.01597,0.01345,0.01294,0.01102,0.00946,1.63262,-0.13676,-0.00302,1.56137,-0.15551,-0.01642,1.69321
t-value,2.34516,5.00295,4.31029,3.57375,3.36951,3.23750,3.08604,2.29496,NaN,-1.81681,-1.09006,NaN,-2.02359,-5.47499,NaN


[1] "Excess Return %"


,Size 1,Size 2,Size 3,Size Avg,Size 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Illiq 1,11.55,1.71,1.14,4.8,-8.88
Illiq 2,2.65,1.42,0.99,1.69,-1.67
Illiq 3,2.05,1.38,6.33,3.25,7.97
Illiq Avg,5.42,1.50,2.82,,-0.86
Illiq 3-1,-7.43,-0.33,7.90,0.05,


[1] "t-stats of Return"


,Size 1,Size 2,Size 3,Size Avg,Size 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Illiq 1,2.184,4.426,3.590,3.4,-1.896
Illiq 2,5.591,3.415,2.408,3.804,-5.562
Illiq 3,4.667,3.322,4.559,4.183,7.579
Illiq Avg,4.148,3.721,3.519,,0.04
Illiq 3-1,-1.591,-1.503,6.959,1.288,


[1] "FFCPS Alpha %"


,Size 1,Size 2,Size 3,Size Avg,Size 3-1
,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Illiq 1,18.91,1.60,1.10,7.2,-15.55
Illiq 2,2.59,1.34,0.95,1.63,-1.64
Illiq 3,2.02,1.29,163.26,55.53,169.32
Illiq Avg,7.84,1.41,55.10,,50.71
Illiq 3-1,-13.68,-0.30,156.14,47.39,


* **Fama-MacBeth regressions of returns on size and Illiq**

In [42]:
data <- list(monthly_sizes, monthly_illiq)
names(data) <- c('Size', 'Illiq' )
perform_fama_macbeth_regression(monthly_returns, data)

Intercept,Size,Illiq
<dbl>,<dbl>,<dbl>
0.01449,-0.00202,0.00169
2.55070,-2.47452,3.22264


# 4. Fama-MacBeth regression

The final step in your analysis will be Fama-MacBeth regression which allows you to  control for all sort variables used in your previous analysis. Examine the effect of the third-sort-variable independently and also using Fama-French factors (MKT, HML, SMB), Momentum and traded liquidity factor. For this task use factors obtained from the web of Kenneth French and Ľuboš Pástor.  

### <span style='background: lightblue'>4.1. With self-calculated factor data  </span>

In [43]:
print("1. Only illiq")
data <- list(monthly_illiq)
names(data) <- c('Illiq')
perform_fama_macbeth_regression(monthly_returns, data)

print("2. illiquidity, size and beta")
data <- list(monthly_illiq,monthly_sizes, monthly_betas)
names(data) <- c('Illiq', 'Size','Beta')
perform_fama_macbeth_regression(monthly_returns, data)

print("3. CAMP Model")
data <- list(monthly_betas)
names(data) <- c('Beta')
perform_fama_macbeth_regression(monthly_returns, data)

print("4. FF3 Model")
data <- list(monthly_betas, monthly_sizes, monthly_bm_ratios)
names(data) <- c('Beta', 'Size', 'Value')
perform_fama_macbeth_regression(monthly_returns, data)

print("5. FFCPS Model")
data <- list(monthly_betas, monthly_sizes, monthly_bm_ratios, 
             monthly_momentum, monthly_illiq )
names(data) <- c('Beta', 'Size', 'Value', 'Mom', 'Illiq')
perform_fama_macbeth_regression(monthly_returns, data)

[1] "1. Only illiq"


Intercept,Illiq
<dbl>,<dbl>
0.00926,0.00163
2.29265,3.52832


[1] "2. illiquidity, size and beta"


Intercept,Illiq,Size,Beta
<dbl>,<dbl>,<dbl>,<dbl>
0.00988,0.00178,-0.00164,0.00295
2.09409,2.83572,-1.91176,0.58555


[1] "3. CAMP Model"


Intercept,Beta
<dbl>,<dbl>
0.00655,0.0047
1.75325,0.9234


[1] "4. FF3 Model"


Intercept,Beta,Size,Value
<dbl>,<dbl>,<dbl>,<dbl>
0.01973,0.0020,-0.00302,-0.00605
4.54365,0.3712,-5.18479,-2.75590


[1] "5. FFCPS Model"


Intercept,Beta,Size,Value,Mom,Illiq
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.01369,-0.00197,-0.00147,-0.00611,-0.00010,0.00194
2.90007,-0.39173,-1.81926,-2.68930,-1.44038,3.10339


### <span style='background: lightblue'>4.2. With downloaded excess return spread factor data  </span>

**Calculate portfolio monthly excess returns, and conbime with factors obtained from the web of Kenneth French and Ľuboš Pástor.**

In [44]:
# Initialize an xts object for storing EW and VW portfolio returns.
return_col <- c('EW', 'VW')
port_returns <- as.xts(
    matrix(
        # first month return is empty
        nrow = nrow(monthly_returns),
        #column 1 is EW, column 2 is VW
        ncol = 2
    ),
    # start from second month
    order.by = index(monthly_returns)
)
names(port_returns) <- return_col
# Iterate over rows, every i is each month 
for (i in 1:nrow(port_returns)) {
    current_month <- index(port_returns[i])
    month_returns <- monthly_returns[current_month]
    month_market_caps <- monthly_market_caps[current_month]
    month_equal_weights <- monthly_equal_weights[current_month]
    # Replace NA market caps with 0s so that such observations have no weights.
    # use markcap for VW
    month_returns[is.na(month_returns)] <- 0    
    month_market_caps[is.na(month_market_caps)] <- 0  
    port_returns[i,1] <- weighted.mean(t(month_returns),
        t(month_equal_weights))
    port_returns[i,2] <- weighted.mean(t(month_returns), 
        t(month_market_caps))
}  
# Combine the samples.
sample <- cbind(round(port_returns, digits = 4), factor_data)
head(sample,3)
tail(sample,3)

                EW      VW  mkt.rf    smb     hml     rf     mom        liq
2007-01-31  0.0288  0.0209  0.0140 0.0014 -0.0070 0.0044  0.0065 0.00375580
2007-02-28 -0.0097 -0.0305 -0.0196 0.0118 -0.0012 0.0038 -0.0004 0.02496826
2007-03-31  0.0069  0.0083  0.0068 0.0015 -0.0094 0.0043  0.0180 0.02931590

                EW      VW  mkt.rf    smb     hml    rf     mom        liq
2020-10-31 -0.0038 -0.0087 -0.0210 0.0440  0.0414 1e-04 -0.0169 0.04065628
2020-11-30  0.1404  0.1121  0.1247 0.0563  0.0210 1e-04 -0.1092 0.02308695
2020-12-31  0.0407  0.0351  0.0463 0.0483 -0.0141 1e-04 -0.0009 0.05024218

**All excess returns from factor data are calculated by value-weighted portfolios, so we just need to calculate value-weighted returns of our portfolio for every month, then do time-series regression with portfolio returns to the excess returns from factors.**

In [45]:
# Create the table of coefficients.
coeffs <- as.data.frame(matrix(nrow = 6 * 2, ncol = 5))
# Specify the names of the columns of the table of coefficients.
names(coeffs) <- c('Illiq','Combine','CAPM','FF3','FFCPS')
attributes(coeffs)$row.names <- c(
    "Intercept", "t.Intercept",
    "liq",  "t.liq",  "mkt", "t.mkt",
    "smb",  "t.smb",  "hml",  "t.hml",
    "mom",  "t.mom"
)
vw_liq <- lm('VW ~ liq', data = sample)
coeffs[c(1, 3), 1]  <- summary(vw_liq)$coefficients[, 1]
coeffs[c(2, 4), 1]  <- summary(vw_liq)$coefficients[, 3]
# combined factors model
vw_comb <- lm('VW ~ liq + mkt.rf + smb', data = sample)
coeffs[c(1, 3, 5, 7), 2]  <- summary(vw_comb)$coefficients[, 1]
coeffs[c(2, 4, 6, 8), 2]  <- summary(vw_comb)$coefficients[, 3]
# CAPM model
vw_capm <- lm('VW ~ mkt.rf', data = sample)
coeffs[c(1, 5), 3]  <- summary(vw_capm)$coefficients[, 1]
coeffs[c(2, 6), 3]  <- summary(vw_capm)$coefficients[, 3]
# FF3 model
vw_ff3 <- lm('VW ~ mkt.rf + smb + hml', data = sample)
coeffs[c(1, 5, 7, 9), 4]  <- summary(vw_ff3)$coefficients[, 1]
coeffs[c(2, 6, 8, 10), 4]  <- summary(vw_ff3)$coefficients[, 3]
# FFCPS model
vw_FFCPS <- lm('VW ~ liq + mkt.rf + smb + hml + mom ', data = sample)
coeffs[c(1, 3, 5, 7, 9, 11), 5]  <- summary(vw_FFCPS)$coefficients[, 1]
coeffs[c(2, 4, 6, 8, 10, 12), 5]  <- summary(vw_FFCPS)$coefficients[, 3]
# Round the results for better readability.
coeffs <- round(coeffs, digits = 4)
coeffs[is.na(coeffs)] <- ""   
summary(vw_FFCPS)
coeffs


Call:
lm(formula = "VW ~ liq + mkt.rf + smb + hml + mom ", data = sample)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0151287 -0.0046529 -0.0006109  0.0041604  0.0227268 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.0048647  0.0005709   8.520 1.05e-14 ***
liq         -0.0379815  0.0155744  -2.439   0.0158 *  
mkt.rf       0.9631219  0.0140180  68.706  < 2e-16 ***
smb         -0.1340383  0.0257668  -5.202 5.89e-07 ***
hml         -0.0092618  0.0226009  -0.410   0.6825    
mom         -0.0254736  0.0184850  -1.378   0.1701    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.00706 on 162 degrees of freedom
Multiple R-squared:  0.9754,	Adjusted R-squared:  0.9747 
F-statistic:  1286 on 5 and 162 DF,  p-value: < 2.2e-16


,Illiq,Combine,CAPM,FF3,FFCPS
,<chr>,<chr>,<chr>,<chr>,<dbl>
Intercept,0.0123,0.0048,0.0048,0.0049,0.0049
t.Intercept,3.6437,8.6203,7.8567,8.4499,8.5203
liq,0.2125,-0.0426,,,-0.0380
t.liq,2.5078,-2.9749,,,-2.4387
mkt,,0.9688,0.9348,0.959,0.9631
t.mkt,,75.6081,71.7077,71.9764,68.7060
smb,,-0.1309,,-0.147,-0.1340
t.smb,,-5.1825,,-5.7344,-5.2020
hml,,,,0.0219,-0.0093


**Market Beta in CAPM is 0.93,and very significant as t-stats 71.7, because our 250 stocks almost represent the market.**